In [ ]:
from keras.models import Sequential
from keras.layers.core import Reshape, Activation, Dropout
from keras.layers import LSTM, merge, Dense

def VQA_MODEL():
    image_feature_size = 2048
    word_feature_size = 300
    number_of_LSTM = 3
    number_of_hidden_units_LSTM = 512
    max_length_questions = 30
    number_of_dense_layers = 4
    number_of_hidden_units = 1024
    activation_function = 'tanh'
    dropout = 0.5


    # Image model
    model_image = Sequential()
    model_image.add(Reshape((image_feature_size,), input_shape=(image_feature_size,)))

    # Language Model
    model_language = Sequential()
    model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=True, input_shape=(max_length_questions, word_feature_size)))
    model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=True))
    model_language.add(LSTM(number_of_hidden_units_LSTM, return_sequences=False))

    # combined model
    model = Sequential()
    model.add(merge.Concatenate([model_language, model_image], axis=1))

    for _ in range(number_of_dense_layers):
        model.add(Dense(number_of_hidden_units, kernel_initializer='uniform'))
        model.add(Activation(activation_function))
        model.add(Dropout(dropout))

    model.add(Dense(1000))
    model.add(Activation('softmax'))

    return model



In [11]:
import json
import h5py
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
from keras.models import Model

In [12]:
data = h5py.File('/home/ubuntu/data/data_processed/data_prepro.h5')

In [19]:
data['ques_train'][0].shape

(26,)

In [ ]:
question_data['ques_length_train'][:-1]

In [ ]:
import json

In [17]:
question_data = json.load(open('/home/ubuntu/data/questions_annotations/vqa_raw_train.json','r'))

In [18]:
question_data[-1]

{'MC_ans': ['fish',
  'no needs more',
  'green',
  'special',
  'white',
  '1',
  'red',
  'yes',
  'not',
  'no',
  'blue',
  '4',
  'low in fat',
  '8691',
  'wool',
  '3',
  '2',
  'east and west'],
 'ans': 'yes',
 'img_path': 'val2014/COCO_val2014_000000552610.jpg',
 'ques_id': 5526102,
 'question': 'Is there water in the water bottle?'}

In [ ]:
def image_preprocess(img_path):
    image = load_img(img_path, target_size=(299, 299))
    image = img_to_array(image)

    # our input image is now represented as a NumPy array of shape
    # (inputShape[0], inputShape[1], 3) however we need to expand the
    # dimension by making the shape (1, inputShape[0], inputShape[1], 3)
    # so we can pass it through thenetwork
    image = np.expand_dims(image, axis=0)

    # pre-process the image using the appropriate function based on the
    # model that has been loaded (i.e., mean subtraction, scaling, etc.)
    image = preprocess_input(image)
    return image

In [ ]:
base_model = Xception(weights='imagenet',input_shape=(299, 299, 3))
image_model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Flatten, Embedding, Merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import h5py

In [ ]:
def Word2VecModel(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate):
    print("Creating text model...")
    model = Sequential()
    model.add(Embedding(num_words, embedding_dim,
        weights=[embedding_matrix], input_length=seq_length, trainable=False))
    model.add(LSTM(output_dim=512, return_sequences=True, input_shape=(seq_length, embedding_dim)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(output_dim=512, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1024, activation='tanh'))
    return model

def img_model(dropout_rate):
    print("Creating image model...")
    model = Sequential()
    model.add(Dense(1024, input_dim=2048, activation='tanh'))
    return model

def vqa_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    vgg_model = img_model(dropout_rate)
    lstm_model = Word2VecModel(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate)
    print("Merging final model...")
    fc_model = Sequential()
    fc_model.add(Merge([vgg_model, lstm_model], mode='mul'))
    fc_model.add(Dropout(dropout_rate))
    fc_model.add(Dense(1000, activation='tanh'))
    fc_model.add(Dropout(dropout_rate))
    fc_model.add(Dense(num_classes, activation='softmax'))
    fc_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
        metrics=['accuracy'])
    return fc_model

In [1]:
import spacy

In [ ]:
def get_question_features(question):
    ''' For a given question, a unicode string, returns the time series vector
    with each word (token) transformed into a 300 dimension representation
    calculated using Glove Vector '''
    word_embeddings = spacy.load('en', vectors='en_glove_cc_300_1m_vectors')
    tokens = word_embeddings(question)
    question_tensor = np.zeros((1, len(tokens), 300))
    for j in xrange(len(tokens)):
            question_tensor[0,j,:] = tokens[j].vector
    return question_tensor

In [16]:
nlp = spacy.load('en', vectors='/home/ubuntu/data_processed/data/glove.6B.300d.txt')

RuntimeError: Language not supported: /home/ubuntu/data_processed/data/glove.6B.300d.txt

In [14]:
obama = nlp(u"obama")
putin = nlp(u"putin")

In [4]:
apples = doc7[0]
oranges = doc7[2]

In [15]:
obama.similarity(putin)

0.0

Python 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
>>> 

In [3]:
with open ('/home/ubuntu/data_processed/data/glove.6B.300d.txt', 'r') as f:
    txt = f.readlines()

In [7]:
import numpy as np

In [8]:
embeddings = {}

for word in txt:
    word = word.split()
    text_word = word[0]
    vector_word = np.array([float(val) for val in word[1:]])
    embeddings[text_word] = vector_word

In [10]:
len(embeddings['obama'])

300